<a href="https://colab.research.google.com/github/mancher07/Stable-Diffusion-3/blob/main/SD3_jupyter_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone -b master https://github.com/camenduru/ComfyUI /content/MasterUI
%cd /content/MasterUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.26.post1
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/adamo1139/stable-diffusion-3-medium-ungated/resolve/main/sd3_medium_incl_clips_t5xxlfp8.safetensors -d /content/MasterUI/model -o sd3_medium_incl_clips_t5xxlfp8.safetensors

import torch
import random
import node_helpers
from comfy.sd import load_checkpoint_guess_config
import nodes
import numpy as np
from PIL import Image

with torch.inference_mode():
    model_patcher, clip, vae, clipvision = load_checkpoint_guess_config("/content/MasterUI/model/sd3_medium_incl_clips_t5xxlfp8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

In [ ]:
# Función para ajustar los valores de conditioning
def zero_out(conditioning):
    c = []
    for t in conditioning:
        d = t[1].copy()
        if "pooled_output" in d:
            d["pooled_output"] = torch.zeros_like(d["pooled_output"])
        n = [torch.zeros_like(t[0]), d]
        c.append(n)
    return (c, )

# Configuración del input
input_config = {
    "cfg": 3.5,
    "image": "https://replicate.delivery/pbxt/LFL6UhoUIvCXhOzy9ndSKI7F7CoR1wzoJrrISDtOUuhcIMjK/IMG_20240710_194648.png",
    "steps": 28,
    "prompt": "A brutal slam/death metal album cover, reflecting a chaotic hell with giant titan octopus with goat skulls, macabre detail, PlayStation 5 graphics, very clear sharpness, shading on all objects, render 4D",
    "aspect_ratio": "1:1",
    "output_format": "png",
    "output_quality": 100,
    "negative_prompt": "verybadimagenegative_v1.3, ng_deepnegative_v1_75t, (ugly face:0.8),cross-eyed,sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, bad anatomy, DeepNegative, facing away, tilted head, {Multiple people}, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot, ((repeating hair))",
    "prompt_strength": 0.5
}

with torch.inference_mode():
    latent = {"samples": torch.ones([1, 16, 1024 // 8, 1024 // 8]) * 0.0609}

    prompt = input_config["prompt"]
    cond, pooled = clip.encode_from_tokens(clip.tokenize(prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]

    negative_prompt = input_config["negative_prompt"]
    n_cond, n_pooled = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
    n_cond = [[n_cond, {"pooled_output": n_pooled}]]

    n_cond1 = node_helpers.conditioning_set_values(n_cond, {"start_percent": 0, "end_percent": 0.1})
    n_cond2 = zero_out(n_cond)
    n_cond2 = node_helpers.conditioning_set_values(n_cond2[0], {"start_percent": 0.1, "end_percent": 1.0})
    n_cond = n_cond1 + n_cond2

    seed = 1
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    sample = nodes.common_ksampler(model=model_patcher,
                            seed=seed,
                            steps=input_config["steps"],
                            cfg=input_config["cfg"],
                            sampler_name="dpmpp_2m",
                            scheduler="sgm_uniform",
                            positive=cond,
                            negative=n_cond,
                            latent=latent,
                            denoise=1)

    sample = sample[0]["samples"].to(torch.float16)
    vae.first_stage_model.cuda()
    decoded = vae.decode_tiled(sample).detach()

image_array = np.array(decoded * 255, dtype=np.uint8)[0]
Image.fromarray(image_array).save("output_image.png", format=input_config["output_format"], quality=input_config["output_quality"])